In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import scipy,scipy.stats
import math
from collections import Counter
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from statsmodels.api import add_constant

In [3]:
%matplotlib inline

In [134]:
pisatrain=pd.read_csv('pisa2009train.csv')

In [7]:
pisatest=pd.read_csv('pisa2009test.csv')

In [23]:
pisatrain.info()
pisatrain.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3663 entries, 0 to 3662
Data columns (total 24 columns):
grade                    3663 non-null int64
male                     3663 non-null int64
raceeth                  3628 non-null object
preschool                3607 non-null float64
expectBachelors          3601 non-null float64
motherHS                 3566 non-null float64
motherBachelors          3266 non-null float64
motherWork               3570 non-null float64
fatherHS                 3418 non-null float64
fatherBachelors          3094 non-null float64
fatherWork               3430 non-null float64
selfBornUS               3594 non-null float64
motherBornUS             3592 non-null float64
fatherBornUS             3550 non-null float64
englishAtHome            3592 non-null float64
computerForSchoolwork    3598 non-null float64
read30MinsADay           3629 non-null float64
minutesPerWeekEnglish    3477 non-null float64
studentsInEnglish        3414 non-null float64
schoo

,grade,male,raceeth,preschool,expectBachelors,motherHS,motherBachelors,motherWork,fatherHS,fatherBachelors,fatherWork,selfBornUS,motherBornUS,fatherBornUS,englishAtHome,computerForSchoolwork,read30MinsADay,minutesPerWeekEnglish,studentsInEnglish,schoolHasLibrary,publicSchool,urban,schoolSize,readingScore
0,11,1,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,225.0,NaN,1.0,1,1,673.0,476.00
1,11,1,White,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,450.0,25.0,1.0,1,0,1173.0,575.01
2,9,1,White,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,250.0,28.0,1.0,1,0,1233.0,554.81
3,10,0,Black,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,200.0,23.0,1.0,1,1,2640.0,458.11
4,10,1,Hispanic,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,250.0,35.0,1.0,1,1,1095.0,613.89


In [15]:
len(pisatrain)

3663

In [36]:
pisatrain.groupby(['male'])['readingScore'].mean()

male
0    512.940631
1    483.532479
Name: readingScore, dtype: float64

In [39]:
pisatrain.isna().sum()

grade                      0
male                       0
raceeth                   35
preschool                 56
expectBachelors           62
motherHS                  97
motherBachelors          397
motherWork                93
fatherHS                 245
fatherBachelors          569
fatherWork               233
selfBornUS                69
motherBornUS              71
fatherBornUS             113
englishAtHome             71
computerForSchoolwork     65
read30MinsADay            34
minutesPerWeekEnglish    186
studentsInEnglish        249
schoolHasLibrary         143
publicSchool               0
urban                      0
schoolSize               162
readingScore               0
dtype: int64

In [135]:
pisatrain1=pisatrain.dropna(axis=0)

In [136]:
pisatest1=pisatest.dropna(axis=0)

In [42]:
pisatrain1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2414 entries, 1 to 3662
Data columns (total 24 columns):
grade                    2414 non-null int64
male                     2414 non-null int64
raceeth                  2414 non-null object
preschool                2414 non-null float64
expectBachelors          2414 non-null float64
motherHS                 2414 non-null float64
motherBachelors          2414 non-null float64
motherWork               2414 non-null float64
fatherHS                 2414 non-null float64
fatherBachelors          2414 non-null float64
fatherWork               2414 non-null float64
selfBornUS               2414 non-null float64
motherBornUS             2414 non-null float64
fatherBornUS             2414 non-null float64
englishAtHome            2414 non-null float64
computerForSchoolwork    2414 non-null float64
read30MinsADay           2414 non-null float64
minutesPerWeekEnglish    2414 non-null float64
studentsInEnglish        2414 non-null float64
schoo

In [43]:
pisatest1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990 entries, 0 to 1569
Data columns (total 24 columns):
grade                    990 non-null int64
male                     990 non-null int64
raceeth                  990 non-null object
preschool                990 non-null float64
expectBachelors          990 non-null float64
motherHS                 990 non-null float64
motherBachelors          990 non-null float64
motherWork               990 non-null float64
fatherHS                 990 non-null float64
fatherBachelors          990 non-null float64
fatherWork               990 non-null float64
selfBornUS               990 non-null float64
motherBornUS             990 non-null float64
fatherBornUS             990 non-null float64
englishAtHome            990 non-null float64
computerForSchoolwork    990 non-null float64
read30MinsADay           990 non-null float64
minutesPerWeekEnglish    990 non-null float64
studentsInEnglish        990 non-null float64
schoolHasLibrary         

In [44]:
pisatrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3663 entries, 0 to 3662
Data columns (total 24 columns):
grade                    3663 non-null int64
male                     3663 non-null int64
raceeth                  3628 non-null object
preschool                3607 non-null float64
expectBachelors          3601 non-null float64
motherHS                 3566 non-null float64
motherBachelors          3266 non-null float64
motherWork               3570 non-null float64
fatherHS                 3418 non-null float64
fatherBachelors          3094 non-null float64
fatherWork               3430 non-null float64
selfBornUS               3594 non-null float64
motherBornUS             3592 non-null float64
fatherBornUS             3550 non-null float64
englishAtHome            3592 non-null float64
computerForSchoolwork    3598 non-null float64
read30MinsADay           3629 non-null float64
minutesPerWeekEnglish    3477 non-null float64
studentsInEnglish        3414 non-null float64
schoo

In [48]:
pisatrain['raceeth'].unique()

array([nan, 'White', 'Black', 'Hispanic', 'Asian', 'More than one race',
       'American Indian/Alaska Native',
       'Native Hawaiian/Other Pacific Islander'], dtype=object)

In [49]:
pisatrain.corr()

,grade,male,preschool,expectBachelors,motherHS,motherBachelors,motherWork,fatherHS,fatherBachelors,fatherWork,selfBornUS,motherBornUS,fatherBornUS,englishAtHome,computerForSchoolwork,read30MinsADay,minutesPerWeekEnglish,studentsInEnglish,schoolHasLibrary,publicSchool,urban,schoolSize,readingScore
grade,1.000000,-0.097677,-0.023262,0.140200,0.005461,0.043171,0.019385,0.046210,0.061713,0.012731,-0.022154,-0.072814,-0.063530,-0.025675,0.112157,0.065008,0.028157,0.085289,-0.026542,-0.063972,0.076277,0.067454,0.286752
male,-0.097677,1.000000,0.014807,-0.078029,0.030586,0.074250,-0.011641,0.017784,0.076257,0.030614,0.022550,0.015107,0.008212,-0.009790,-0.015485,-0.220103,0.010700,-0.025404,0.032114,-0.075466,0.036377,0.012921,-0.153928
preschool,-0.023262,0.014807,1.000000,0.076046,0.141552,0.151832,0.069249,0.117443,0.141418,0.065079,0.092893,0.111684,0.112208,0.120480,0.076612,-0.008649,-0.015110,-0.027114,0.036506,-0.070919,-0.053684,-0.032582,0.040537
expectBachelors,0.140200,-0.078029,0.076046,1.000000,0.104886,0.168330,0.069366,0.156028,0.199217,0.003122,-0.010330,0.012359,0.015281,0.044169,0.130413,0.135002,0.018364,0.050988,0.072164,-0.112698,-0.004022,0.016164,0.360144
motherHS,0.005461,0.030586,0.141552,0.104886,1.000000,0.252322,0.176354,0.500723,0.227148,0.038105,0.179856,0.343146,0.336373,0.374486,0.187951,0.011641,0.019021,-0.036948,0.012129,-0.067646,-0.117988,-0.084460,0.156488
motherBachelors,0.043171,0.074250,0.151832,0.168330,0.252322,1.000000,0.142306,0.197929,0.543581,0.071973,0.044804,0.146741,0.125661,0.169733,0.146681,0.021138,0.023004,-0.041765,0.022990,-0.161796,-0.048131,-0.011638,0.214240
motherWork,0.019385,-0.011641,0.069249,0.069366,0.176354,0.142306,1.000000,0.102632,0.057245,0.039578,0.065549,0.096057,0.096592,0.128595,0.083805,-0.049956,0.009103,-0.032115,-0.011439,-0.036711,-0.012350,-0.039499,0.043401
fatherHS,0.046210,0.017784,0.117443,0.156028,0.500723,0.197929,0.102632,1.000000,0.280287,0.100620,0.139385,0.279932,0.308128,0.324131,0.173284,0.032457,0.041421,-0.007217,0.030027,-0.080654,-0.098083,-0.080115,0.175464
fatherBachelors,0.061713,0.076257,0.141418,0.199217,0.227148,0.543581,0.057245,0.280287,1.000000,0.106328,0.011592,0.081497,0.091718,0.138824,0.174256,0.047728,0.007736,-0.019529,0.017605,-0.176680,-0.018506,0.019602,0.266236
fatherWork,0.012731,0.030614,0.065079,0.003122,0.038105,0.071973,0.039578,0.100620,0.106328,1.000000,-0.028350,-0.008205,-0.004942,0.022521,0.078806,-0.012103,0.030462,-0.000204,-0.012325,-0.023248,-0.039698,-0.023414,0.070423


In [54]:
pisatrain['raceeth'].nunique()

7

In [137]:
dummy=pd.get_dummies(pisatrain1['raceeth'],drop_first=True)

In [110]:
pisatrain1.head()

,grade,male,raceeth,preschool,expectBachelors,motherHS,motherBachelors,motherWork,fatherHS,fatherBachelors,fatherWork,selfBornUS,motherBornUS,fatherBornUS,englishAtHome,computerForSchoolwork,read30MinsADay,minutesPerWeekEnglish,studentsInEnglish,schoolHasLibrary,publicSchool,urban,schoolSize,readingScore
1,11,1,White,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,450.0,25.0,1.0,1,0,1173.0,575.01
3,10,0,Black,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,200.0,23.0,1.0,1,1,2640.0,458.11
4,10,1,Hispanic,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,250.0,35.0,1.0,1,1,1095.0,613.89
7,10,0,White,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,300.0,30.0,1.0,1,0,1913.0,439.36
9,10,1,More than one race,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,294.0,24.0,1.0,1,0,899.0,465.90


In [138]:
pisatrain1_dummy=pd.concat([pisatrain1,dummy],axis=1)

In [112]:
type(pisatrain1_dummy)

pandas.core.frame.DataFrame

In [139]:
pisatrain1_dummy=pisatrain1_dummy.drop('raceeth',axis=1)

In [114]:
pisatrain1_dummy.head()

,grade,male,preschool,expectBachelors,motherHS,motherBachelors,motherWork,fatherHS,fatherBachelors,fatherWork,selfBornUS,motherBornUS,fatherBornUS,englishAtHome,computerForSchoolwork,read30MinsADay,minutesPerWeekEnglish,studentsInEnglish,schoolHasLibrary,publicSchool,urban,schoolSize,readingScore,Asian,Black,Hispanic,More than one race,Native Hawaiian/Other Pacific Islander,White
1,11,1,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,450.0,25.0,1.0,1,0,1173.0,575.01,0,0,0,0,0,1
3,10,0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,200.0,23.0,1.0,1,1,2640.0,458.11,0,1,0,0,0,0
4,10,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,250.0,35.0,1.0,1,1,1095.0,613.89,0,0,1,0,0,0
7,10,0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,300.0,30.0,1.0,1,0,1913.0,439.36,0,0,0,0,0,1
9,10,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,294.0,24.0,1.0,1,0,899.0,465.90,0,0,0,1,0,0


In [140]:
x=pisatrain1_dummy.drop('readingScore',axis=1)

In [141]:
y=pisatrain1_dummy['readingScore']

In [142]:
lm=LinearRegression()

In [143]:
lm.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [144]:
print('intercept:',lm.intercept_,'coefficeints',lm.coef_)

intercept: 76.48900632875143 coefficeints [ 2.95427071e+01 -1.45216535e+01 -4.46367032e+00  5.52670801e+01
  6.05877432e+00  1.26380678e+01 -2.80910060e+00  4.01821353e+00
  1.69297547e+01  5.84279834e+00 -3.80627792e+00 -8.79815308e+00
  4.30699432e+00  8.03568537e+00  2.25002323e+01  3.48719242e+01
  1.27884515e-02 -2.86630774e-01  1.22150850e+01 -1.68574750e+01
 -1.10131762e-01  6.54038148e-03  6.31670019e+01  2.64980050e-01
  2.83018415e+01  5.03548051e+01  6.21757262e+01  6.72773271e+01]


In [145]:
pred=lm.predict(x)

In [146]:
r2_score(y,pred)

0.3251433559213568

In [147]:
metrics.mean_squared_error(y,pred)

5382.5041370658955

In [148]:
np.sqrt(metrics.mean_squared_error(y,pred))

73.3655514329845

In [164]:
pisatrain1['raceeth'].unique()

array(['White', 'Black', 'Hispanic', 'More than one race',
       'American Indian/Alaska Native', 'Asian',
       'Native Hawaiian/Other Pacific Islander'], dtype=object)

In [165]:
x2=add_constant(x)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [167]:
lm=sm.OLS(y,x2)

In [168]:
lm2=lm.fit()

In [169]:
lm2.pvalues

const                                     4.042509e-02
grade                                     2.468118e-23
male                                      4.416601e-06
preschool                                 2.005164e-01
expectBachelors                           1.059226e-36
motherHS                                  3.200122e-01
motherBachelors                           1.079791e-03
motherWork                                4.251670e-01
fatherHS                                  4.714698e-01
fatherBachelors                           2.346477e-05
fatherWork                                1.839343e-01
selfBornUS                                6.033074e-01
motherBornUS                              1.818211e-01
fatherBornUS                              4.917762e-01
englishAtHome                             2.415272e-01
computerForSchoolwork                     8.188776e-05
read30MinsADay                            4.489941e-24
minutesPerWeekEnglish                     2.326444e-01
studentsIn

In [170]:
print(lm2.summary())

                            OLS Regression Results                            
Dep. Variable:           readingScore   R-squared:                       0.325
Model:                            OLS   Adj. R-squared:                  0.317
Method:                 Least Squares   F-statistic:                     41.04
Date:                Mon, 16 Sep 2019   Prob (F-statistic):          1.72e-180
Time:                        03:47:33   Log-Likelihood:                -13795.
No. Observations:                2414   AIC:                         2.765e+04
Df Residuals:                    2385   BIC:                         2.781e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

In [178]:
dummy2=pd.get_dummies(pisatest1['raceeth'],drop_first=True)

In [180]:
pisatest1=pd.concat([pisatest1,dummy2],axis=1)

In [189]:
x_test=pisatest1.drop(['readingScore','raceeth'],axis=1)

In [191]:
y_test=pisatest1['readingScore']

In [194]:
lmtest=LinearRegression()

In [195]:
lmtest.fit(x_test,y_test)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [196]:
pred_test=lmtest.predict(x_test)

In [228]:
(pred_test.max())-(pred_test.min())
len(pred_test)

990

In [229]:
(sum(((pred_test)-(y_test))**2)/990)**0.5


74.70434728222544

In [224]:
np.sqrt(metrics.mean_squared_error(y_test,pred_test))

74.70434728222536

In [225]:
metrics.mean_squared_error(y_test,pred_test)*990

5524932.107834697

In [226]:
len(pred_test)

990

In [207]:
np.mean(y)

517.9628873239429

In [230]:
sum(((y_test)-np.mean(y))**2)

7802354.077613839

In [231]:
r2_score(y_test,pred_test)

0.29156410673966104

In [212]:
pisatrain.groupby(['grade'])['readingScore'].mean()

grade
8     389.970000
9     412.826398
10    503.256963
11    525.996915
12    569.280000
Name: readingScore, dtype: float64